# Preparativos

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download zaheenhamidani/ultimate-spotify-tracks-db

 58% 9.00M/15.5M [00:00<00:00, 20.1MB/s]
100% 15.5M/15.5M [00:00<00:00, 34.9MB/s]


In [ ]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# el train que nos dieron los profes
df_train_original= pd.read_csv('/content/SpotifyFeatures_train.csv')
df_train = df_train_original.copy()
# el dataset nuestro
df_full = pd.read_csv('/content/ultimate-spotify-tracks-db.zip')


# Exploracion de datos

In [ ]:
df_train.head()

,Unnamed: 0,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,107360,Opera,Richard Wagner,Gotterdammerung (Twilight of the Gods): Prolog...,32pnRCr5u3nzO55MgoEJwV,6,0.89900,0.147,131787,0.016,0.947000,B,0.0889,-24.317,Major,0.0380,137.591,4/4,0.0292
1,165017,Reggaeton,Ronald El Killa,Esta Pena,3uMQ7q5oU0VgYl9SyCF2MR,27,0.00613,0.857,255134,0.599,0.000085,B,0.2090,-5.677,Major,0.0951,96.022,4/4,0.5860
2,141044,Indie,LP,Recovery,1a5P7IvLUhRhLnnbkev3PI,59,0.27400,0.518,234905,0.545,0.000023,A#,0.1830,-7.893,Major,0.0385,75.060,4/4,0.1620
3,141709,Indie,The National,Carin at the Liquor Store,4maI5FwVIzjrErduPjjZLO,53,0.59400,0.540,213733,0.615,0.803000,A#,0.1200,-9.264,Major,0.0340,93.170,3/4,0.2390
4,106321,Opera,Renata Tebaldi,Giordano: Andrea Chenier: Come un bel di di ma...,5VEVnqQu6cFewcbWfbkUeE,8,0.95500,0.363,170240,0.196,0.000115,C#,0.6840,-15.675,Minor,0.0452,63.186,4/4,0.2510


In [ ]:
df_full.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [ ]:
df_train.columns

Index(['Unnamed: 0', 'genre', 'artist_name', 'track_name', 'track_id',
       'popularity', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence'],
      dtype='object')

In [ ]:
df_train.dtypes

Unnamed: 0            int64
genre                object
artist_name          object
track_name           object
track_id             object
popularity            int64
acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
instrumentalness    float64
key                  object
liveness            float64
loudness            float64
mode                 object
speechiness         float64
tempo               float64
time_signature       object
valence             float64
dtype: object

# Limpieza de datos

In [ ]:
df_train = df_train.drop(['Unnamed: 0'], axis=1)


In [ ]:

indexNames = df_train[ df_train['genre'] == 'Comedy' ].index
df_train.drop(indexNames , inplace=True)

In [ ]:
df_train['genre'].value_counts()

Soundtrack          7758
Jazz                7634
Indie               7603
Children’s Music    7554
Pop                 7532
Electronic          7500
Rock                7455
Hip-Hop             7422
Folk                7406
Classical           7395
Rap                 7367
Alternative         7342
Soul                7280
R&B                 7242
World               7211
Blues               7187
Ska                 7154
Reggaeton           7139
Anime               7131
Reggae              7019
Dance               6950
Country             6861
Opera               6612
Movie               6261
Children's Music    4298
A Capella             96
Name: genre, dtype: int64

In [ ]:
#columnas con datos faltantes
cols_with_missing = [col for col in df_train.columns if df_train[col].isnull().any()] 
if len(cols_with_missing)==0:
  print('no faltan valores') 

no faltan valores


In [ ]:
# saco las filas correspondientes
print(cols_with_missing)
df_train.dropna(axis=0, inplace=True)


[]


# Variables Categoricas

In [ ]:
# Variable categoricas
s = (df_train.dtypes == 'object')
object_cols = list(s[s].index)
print(object_cols)

['genre', 'artist_name', 'track_name', 'track_id', 'key', 'mode', 'time_signature']


In [ ]:
list_of_timeSignature = df_train['time_signature'].unique()
print(list_of_timeSignature)

['4/4' '3/4' '5/4' '1/4' '0/4']


In [ ]:
df_train['time_signature'].value_counts()

4/4    156851
3/4     16544
5/4      3357
1/4      1651
0/4         6
Name: time_signature, dtype: int64

Decidimos no utilizar Time_signature como feature por varios motivos


*   En musica no existe compaces de 1/4 ni de 0/4 entre ambos suman 1656 valores lo que nos parecio un numero considerable
*   En una cancion puede haber multiples cambios de compaces, en la musica progresiva es algo muy comun

*   La mayoria de la musica se encuentra en 4/4 podemos ver que hay 156851 valores que considerablemente mayor a lo suma de todas las otras categorias esto introduceria un gran sesgo en nuestro modelo 










In [ ]:
df_train = df_train.drop(['time_signature'], axis=1)

In [ ]:
# variables categoricas con baja cardinalidad
categorical_cols = [cname for cname in df_train.columns if
                    df_train[cname].nunique() < 14 and 
                    df_train[cname].dtype == "object"]
print(categorical_cols)

['key', 'mode']


Quedan descartadas las variables categoricas: 

*   Time_signature por los motivos comentados anteriormente
*   track_id: no hay ninguna razon para utilizarla
*   el nombre del artista: son demasiadas variables de momento las sacamos pero es cierto que es un feature interesente 
*   track_name: todos sus valores son distintos 







In [ ]:
list_of_modes = df_train['mode'].unique()
print(list_of_modes)

['Major' 'Minor']


In [ ]:
list_of_keys = df_train['key'].unique()
print(list_of_keys)


['B' 'A#' 'C#' 'C' 'F#' 'E' 'A' 'D' 'G' 'F' 'G#' 'D#']


# Removiendo datos de columnas faltantes, separacion en entrenamiento y validacion

In [ ]:
# Remove rows with missing target, separate target from predictors

X = df_train.drop(columns=['genre'])
y = df_train['genre']


In [ ]:
from sklearn.model_selection import train_test_split
X_train_full, X_valid_full, y_train, y_valid = train_test_split(df_train, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [ ]:
y_train.value_counts().sort_index()

A Capella             75
Alternative         5871
Anime               5734
Blues               5733
Children's Music    3459
Children’s Music    6086
Classical           5914
Country             5447
Dance               5549
Electronic          5993
Folk                5899
Hip-Hop             5917
Indie               6113
Jazz                6098
Movie               5012
Opera               5252
Pop                 6061
R&B                 5807
Rap                 5845
Reggae              5611
Reggaeton           5716
Rock                5971
Ska                 5702
Soul                5853
Soundtrack          6231
World               5778
Name: genre, dtype: int64

In [ ]:
y_valid.value_counts().sort_index()

A Capella             21
Alternative         1471
Anime               1397
Blues               1454
Children's Music     839
Children’s Music    1468
Classical           1481
Country             1414
Dance               1401
Electronic          1507
Folk                1507
Hip-Hop             1505
Indie               1490
Jazz                1536
Movie               1249
Opera               1360
Pop                 1471
R&B                 1435
Rap                 1522
Reggae              1408
Reggaeton           1423
Rock                1484
Ska                 1452
Soul                1427
Soundtrack          1527
World               1433
Name: genre, dtype: int64

In [ ]:

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

print(numerical_cols)

['popularity', 'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']


In [ ]:
#Seleccion de columnas
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()


# Un poco de ingenieria de datos

In [ ]:
from sklearn.preprocessing import OrdinalEncoder


label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

ordinal_encoder = OrdinalEncoder()
label_X_train[categorical_cols] = ordinal_encoder.fit_transform(X_train[categorical_cols])
label_X_valid[categorical_cols] = ordinal_encoder.transform(X_valid[categorical_cols])




# Probando modelos

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(multi_class = 'multinomial', solver='lbfgs', max_iter=200)

model.fit(label_X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = model.predict(label_X_valid)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_valid, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                  precision    recall  f1-score   support

       A Capella       0.00      0.00      0.00        21
     Alternative       0.10      0.07      0.08      1471
           Anime       0.31      0.31      0.31      1397
           Blues       0.00      0.00      0.00      1454
Children's Music       0.21      0.45      0.29       839
Children’s Music       0.09      0.10      0.10      1468
       Classical       0.33      0.33      0.33      1481
         Country       0.08      0.13      0.10      1414
           Dance       0.14      0.00      0.00      1401
      Electronic       0.12      0.17      0.14      1507
            Folk       0.12      0.07      0.09      1507
         Hip-Hop       0.10      0.07      0.08      1505
           Indie       0.10      0.02      0.04      1490
            Jazz       0.11      0.18      0.14      1536
           Movie       0.47      0.10      0.17      1249
           Opera       0.33      0.57      0.42      1360
             

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=30, random_state=42)
rfc.fit(label_X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
y_rfc = rfc.predict(label_X_valid)
print(classification_report(y_valid, y_rfc))


                  precision    recall  f1-score   support

       A Capella       1.00      0.14      0.25        21
     Alternative       0.15      0.13      0.14      1471
           Anime       0.59      0.56      0.58      1397
           Blues       0.36      0.37      0.37      1454
Children's Music       0.72      0.74      0.73       839
Children’s Music       0.04      0.05      0.05      1468
       Classical       0.63      0.62      0.62      1481
         Country       0.37      0.38      0.38      1414
           Dance       0.12      0.10      0.11      1401
      Electronic       0.51      0.52      0.52      1507
            Folk       0.19      0.19      0.19      1507
         Hip-Hop       0.15      0.17      0.16      1505
           Indie       0.06      0.06      0.06      1490
            Jazz       0.36      0.35      0.35      1536
           Movie       0.58      0.52      0.55      1249
           Opera       0.77      0.85      0.81      1360
             

## GridSerchCv

In [ ]:
# Numero de arboles
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
# Numero de features considerado al dividir
max_features = ['auto', 'sqrt']
# numero maximo de niveles
max_depth = [2,4]
# Numero minimo de niveles por nodo
min_samples_split = [2, 5]
# Numero minimo de niveles por hoja
min_samples_leaf = [1, 2]
# metodo de seleccion por arbol 
bootstrap = [True, False]

In [ ]:
# Grilla
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


In [ ]:
from sklearn.model_selection import GridSearchCV
rfc_Grid = GridSearchCV(estimator = rfc, param_grid = param_grid, cv = 3, verbose=2, n_jobs = 4)

In [ ]:
rfc_Grid.fit(label_X_train, y_train)


Fitting 3 folds for each of 320 candidates, totalling 960 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  5.2min
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed: 14.4min
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed: 27.2min
[Parallel(n_jobs=4)]: Done 960 out of 960 | elapsed: 47.3min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=30, n_jobs=None,
                                              oob_score=False, random_state=42,
                                   

NameError: ignored